# Linear sub-sampling

In [88]:
import torch
sub_sampling_factor = 4
hidden_dim = 1

Input

In [89]:
x = torch.arange(32).reshape(2, 8, 2)
x

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5],
         [ 6,  7],
         [ 8,  9],
         [10, 11],
         [12, 13],
         [14, 15]],

        [[16, 17],
         [18, 19],
         [20, 21],
         [22, 23],
         [24, 25],
         [26, 27],
         [28, 29],
         [30, 31]]], device='cuda:0')

In [106]:
y = torch.reshape(x, (2, int(8/sub_sampling_factor), 2*sub_sampling_factor))
y.shape

torch.Size([2, 2, 8])

Chunking the input according to the sub-sampling factor

In [95]:
num_chunks = int(x.shape[1] / sub_sampling_factor)
chunked_c = torch.chunk(x, num_chunks, dim=1)
print(f'chunked input: {chunked_c}')
print(chunked_c[0].shape)


chunked input: (tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5],
         [ 6,  7]],

        [[16, 17],
         [18, 19],
         [20, 21],
         [22, 23]]], device='cuda:0'), tensor([[[ 8,  9],
         [10, 11],
         [12, 13],
         [14, 15]],

        [[24, 25],
         [26, 27],
         [28, 29],
         [30, 31]]], device='cuda:0'))
torch.Size([2, 4, 2])


Concatenating along the time axis for the first chuck

In [102]:
cl = [torch.reshape(chunk, (2, 1, -1)) for chunk in chunked_c]
cl
# c = torch.FloatTensor(cl)

[tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7]],
 
         [[16, 17, 18, 19, 20, 21, 22, 23]]], device='cuda:0'),
 tensor([[[ 8,  9, 10, 11, 12, 13, 14, 15]],
 
         [[24, 25, 26, 27, 28, 29, 30, 31]]], device='cuda:0')]

In [91]:
concatenated_chunk = torch.reshape(chunked_c[0], (2, 1, -1))
concatenated_chunk = torch.tensor(concatenated_chunk,
                                  dtype=torch.long)
print(concatenated_chunk)
concatenated_chunk.shape

tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7]],

        [[16, 17, 18, 19, 20, 21, 22, 23]]])


/home/dk/Desktop/projects/audio-captioning-lm/.env-audio-cap-lm/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


torch.Size([2, 1, 8])

Make the learn-able sub-sampling using linear layer

In [92]:
from torch.nn import Linear, Sequential, LeakyReLU
linear_subsampling = Sequential(
    Linear(in_features=8,
           out_features=2),
    LeakyReLU()
    )
linear_subsampling

Sequential(
  (0): Linear(in_features=8, out_features=2, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
)

In [93]:
sub_sampled_chunk = linear_subsampling(concatenated_chunk)



RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_mm